In [ ]:
%pip install -r requirements.txt

Basic Anthropic Integration

In [ ]:
import anthropic
import json
import random
import string

#TODO: Remove keys
anthropic_key = "YOUR_ANTHROPIC_KEY"
payi_api_key = "YOUR_PAYI_API_KEY"

payi_headers = {
    "xProxy-Forward-x-api-key": anthropic_key,
    "xProxy-Forward-anthropic-version": "2023-06-01"
}

#TODO: Replace with Prod endpoint under the DNS
payi_base_url = "https://apim-payi-qa-eastus.azure-api.net/api/v1/proxy/anthropic/"

client = anthropic.Anthropic(
    base_url = payi_base_url,
    auth_token = payi_api_key,
    default_headers=payi_headers
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ]
)
completion = message.content[0].text
print(completion)

proxy_result = message.proxy_result
print(json.dumps(proxy_result, indent=4))


In [ ]:
#TODO: Merge into the library directly
from typing import List, Dict

def create_budget_header_from_ids(budget_ids: List[str]) -> Dict[str, str]:
    if not isinstance(budget_ids, list):
        raise TypeError("budget_ids must be a list")
    
    valid_ids = [id.strip() for id in budget_ids if isinstance(id, str) and id.strip()]
    if valid_ids:
        return {"xProxy-Budget-IDs": ",".join(valid_ids)}
    return {}

def create_request_header_from_ids(request_ids: List[str]) -> Dict[str, str]:
    if not isinstance(request_ids, list):
        raise TypeError("request_tags must be a list")
    
    valid_ids = [id.strip() for id in request_ids if isinstance(id, str) and id.strip()]
    if valid_ids:
        return {"xProxy-Request-Tags": ",".join(valid_ids)}
    return {}

def create_headers(budget_ids: List[str] = None, request_tags: List[str] = None) -> Dict[str, str]:
    headers = {}
    if budget_ids is not None:
        headers.update(create_budget_header_from_ids(budget_ids))
    if request_tags is not None:
        headers.update(create_request_header_from_ids(request_tags))
    return headers

Use the Pay-i SDK to send a request with a request tag

In [ ]:
from payi import Payi
#from payi.lib.helpers import create_headers

#TODO: Replace with production endpoint
payi_client = Payi(
    payi_api_key=payi_api_key,
    base_url="https://apim-payi-qa-eastus.azure-api.net"
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a request tag test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = message.content[0].text
print(completion)

proxy_result = message.proxy_result
print(json.dumps(proxy_result, indent=4))

Create a budget and make a request with that budget

In [ ]:
#Create a budget
budget_response = payi_client.budgets.create(
    #create a random name for the demo
    budget_name=''.join(random.choices(string.ascii_letters + string.digits, k=6)), 
    max=12.50, #$12.50 USD
    budget_type="Liberal",
    budget_response_type="Allow",
    budget_tags=["example_budget"]
)

budget_name = budget_response.budget.budget_name
budget_id = budget_response.budget.budget_id

print("Budget Created")
print(f"Budget Name: {budget_name}")
print(f"Budget ID: {budget_id}")

#Make a request using the new budget and request tags
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a request tag test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        budget_ids=[budget_id]
    )
)

completion = message.content[0].text
print(completion)

proxy_result = message.proxy_result
print(json.dumps(proxy_result, indent=4))

See budget status

In [ ]:
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input_cost.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output_cost.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total_cost.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.anthropic",
    resource="claude-3-5-sonnet-20240620",
    input=50,
    output=100,
    budget_ids= ",".join([budget_id]),
    request_tags = ",".join(["a", "b"])
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.cost.input.base}")
print(f"Output Base Cost: {response.cost.output.base}")
print(f"Total Base Cost: {response.cost.total.base}")

Reset a budget back to zero tracked cost

In [11]:
response = payi_client.budgets.reset(budget_id=budget_id)
print(response.message)
print("State prior to reset: ")
print(f"Budget Name: {response.budget_history.budget_name}")
print(f"Budget ID: {response.budget_history.budget_id}")
print(f"Budget Tags: {response.budget_history.budget_tags}")
print(f"Budget Reset Timestamp: {response.budget_history.reset_date}")
print(f"Budget Input Base Cost: {response.budget_history.totals.cost.input.base}")
print(f"Budget Output Base Cost: {response.budget_history.totals.cost.output.base}")
print(f"Budget Total Base Cost: {response.budget_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.budgets.retrieve(budget_id=budget_id)
print(f"Budget Name: {response.budget.budget_name}")
print(f"Budget ID: {response.budget.budget_id}")
print(f"Budget Creation Timestamp: {response.budget.budget_creation_timestamp}")
print(f"Budget Tags: {response.budget.budget_tags}")
print(f"Budget Input Base Cost: {response.budget.totals.cost.input_cost.base}")
print(f"Budget Output Base Cost: {response.budget.totals.cost.output_cost.base}")
print(f"Budget Total Base Cost: {response.budget.totals.cost.total_cost.base}")

Budget has been successfully reset. Budget History item created
State prior to reset: 
Budget Name: hQ4f0o
Budget ID: b69c9dbb-fb3c-4799-45b9-08dc97a2c1c2
Budget Tags: ['example_budget']
Budget Reset Timestamp: 2024-06-29 21:40:25.586807+00:00
Budget Input Base Cost: 5.1e-05
Budget Output Base Cost: 0.000135
Budget Total Base Cost: 0.000186

State after reset:
Budget Name: hQ4f0o
Budget ID: b69c9dbb-fb3c-4799-45b9-08dc97a2c1c2
Budget Creation Timestamp: 2024-06-29 21:39:02.564307
Budget Tags: ['example_budget']
Budget Input Base Cost: 0.0
Budget Output Base Cost: 0.0
Budget Total Base Cost: 0.0


List and then delete all budgets

In [12]:
response = payi_client.budgets.list()
for budget in response.items:
    print("Deleting budget with id:" + budget.budget_id)
    payi_client.budgets.delete(budget.budget_id)

Deleting budget with id:b69c9dbb-fb3c-4799-45b9-08dc97a2c1c2
